# Primitive di Rete e Analisi di Header: Edizione Avanzata

**Obiettivo:** Questo notebook è pensato per studenti del quinto anno di un istituto tecnico informatico e mira a consolidare le conoscenze sul livello di trasporto, analizzando in dettaglio le primitive dei socket, concetti avanzati come il controllo di congestione e la chiusura delle connessioni, e l'analisi programmatica di pacchetti di rete.

## 1. Teoria: Servizi del Livello di Trasporto e Protocolli (TCP vs. UDP)

Il livello di trasporto fornisce servizi di comunicazione end-to-end tra applicazioni in esecuzione su host diversi. I due protocolli principali sono TCP (Transmission Control Protocol) e UDP (User Datagram Protocol).

| Servizio | TCP (Transmission Control Protocol) | UDP (User Datagram Protocol) |
|---|---|---|
| **Connessione** | Orientato alla connessione (three-way handshake) | Senza connessione (connectionless) |
| **Affidabilità** | Elevata: consegna garantita, ordinata e senza errori | Bassa: nessuna garanzia su consegna, ordine o duplicati |
| **Controllo di Flusso**| Sì (finestra di scorrimento) | No |
| **Controllo di Congestione**| Sì (es. Slow Start, Congestion Avoidance) | No |
| **Overhead** | Alto (header di 20-60 byte) | Basso (header di 8 byte) |
| **Casi d'uso** | Web (HTTP/S), email (SMTP), trasferimento file (FTP) | Streaming video/audio, gaming online, DNS, DHCP |


## 2. Le Primitive dei Socket: La Cassetta degli Attrezzi della Rete

Le primitive dei socket sono le funzioni di base che le applicazioni usano per creare e gestire le comunicazioni di rete. Ecco le principali:

*   `socket()`: Crea un nuovo socket, specificando la famiglia di indirizzi (es. `AF_INET` per IPv4) e il tipo di socket (es. `SOCK_STREAM` per TCP, `SOCK_DGRAM` per UDP).
*   `bind()`: Associa un socket a un indirizzo IP e a una porta specifici. È tipicamente usato sul lato server.
*   `listen()`: Mette il server in ascolto di connessioni in entrata su un socket TCP. Specifica la dimensione della coda di connessioni pendenti.
*   `connect()`: Usato dal client per stabilire una connessione con un server a un indirizzo e porta specifici.
*   `accept()`: Usato dal server per accettare una connessione in entrata. Restituisce un nuovo socket per comunicare con il client e l'indirizzo del client.
*   `send()` / `sendto()`: Inviano dati. `send()` è usato per socket connessi (TCP), mentre `sendto()` è usato per socket non connessi (UDP) e richiede di specificare l'indirizzo di destinazione.
*   `recv()` / `recvfrom()`: Ricevono dati. `recv()` è per socket connessi (TCP), `recvfrom()` per socket non connessi (UDP) e restituisce anche l'indirizzo del mittente.
*   `close()`: Chiude la connessione del socket. Rilascia le risorse associate.
*   `shutdown()`: Offre un controllo più granulare sulla chiusura di una connessione TCP, permettendo di chiudere solo la lettura, solo la scrittura o entrambe.

In [ ]:
# Esempio: Echo Server TCP
# Questo server si mette in ascolto sulla porta 12345 e rimanda al client qualsiasi dato riceve.
# Per eseguirlo in un notebook, dovreste lanciarlo in un terminale separato o come processo in background.

import socket

HOST = '127.0.0.1'  # Indirizzo di loopback
PORT = 12345        # Porta non privilegiata

def start_echo_server():
    print("Avvio l'echo server...")
    with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
        s.bind((HOST, PORT))
        s.listen()
        print(f"Server in ascolto su {HOST}:{PORT}")
        conn, addr = s.accept()
        with conn:
            print(f"Connesso da {addr}")
            while True:
                data = conn.recv(1024)
                if not data:
                    break
                print(f"Ricevuto: {data.decode('utf-8')}")
                conn.sendall(data) # Rimanda i dati al client
                print(f"Inviato: {data.decode('utf-8')}")
    print("Server terminato.")

# Per eseguire questo codice, potreste chiamare start_echo_server() da uno script.
# Dentro Colab, l'esecuzione di un server bloccante non è ideale.
print("Cella contenente il codice del server. Non eseguire direttamente qui.")

In [ ]:
# Esempio: Client per l'Echo Server
# Per eseguire questo esempio: avvia il server in un terminale, poi esegui questa cella.

import socket

HOST = '127.0.0.1'
PORT = 12345

try:
    with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
        s.connect((HOST, PORT))
        messaggio = b'Ciao, mondo!'
        s.sendall(messaggio)
        print(f"Inviato: {messaggio.decode('utf-8')}")
        data = s.recv(1024)
    print(f"Ricevuto (echo): {data.decode('utf-8')}")
except ConnectionRefusedError:
    print("Errore: Connessione rifiutata. Assicurati che il server sia in esecuzione.")

## 3. Teoria Avanzata: Gestione della Connessione e del Traffico

### Confronto tra `close()` e `shutdown()`

Mentre `close()` chiude immediatamente il socket da entrambi i lati (lettura e scrittura), `shutdown()` offre un controllo più fine. È possibile chiudere la connessione in una sola direzione:

*   `shutdown(socket.SHUT_RD)`: Il socket non può più ricevere dati (la scrittura è ancora permessa).
*   `shutdown(socket.SHUT_WR)`: Il socket non può più inviare dati. Il ricevente riceverà una notifica di fine trasmissione (un `recv()` restituirà 0 byte), utile per segnalare che non si invieranno più dati.
*   `shutdown(socket.SHUT_RDWR)`: Equivalente a chiudere in entrambe le direzioni, simile a `close()`.

**Scenario d'uso:** `shutdown(SHUT_WR)` è utile quando un client ha finito di inviare una richiesta a un server e vuole segnalarlo, ma ha ancora bisogno di leggere la risposta completa del server.

### Controllo di Congestione TCP e Slow Start

Il controllo di congestione è il meccanismo con cui TCP evita di sovraccaricare la rete. Una delle fasi iniziali è il **Slow Start**.

Quando una connessione TCP inizia, il mittente non conosce la capacità della rete. Invece di inviare una grande quantità di dati subito (rischiando di creare congestione), inizia con una piccola **finestra di congestione** (congestion window, `cwnd`), tipicamente da 1 a 10 MSS (Maximum Segment Size). Per ogni ACK ricevuto, la `cwnd` viene incrementata (solitamente raddoppia), portando a una crescita esponenziale della quantità di dati inviati. Questo spiega perché la "finestra iniziale" è limitata: è una misura di sicurezza per sondare la rete e aumentare la velocità di trasmissione solo se la rete lo permette.

## 4. Attività Pratica 1: Cattura di Pacchetti

Per analizzare il traffico, è necessario prima catturarlo. Ecco come farlo sui principali sistemi operativi.

### Linux Mint (con `tcpdump`)

`tcpdump` è un potente strumento a riga di comando. Per catturare il traffico e salvarlo in un file:

1.  Apri un terminale.
2.  Identifica l'interfaccia di rete (es. `eth0`, `wlan0`) con il comando `ip a`.
3.  Esegui il comando per catturare il traffico. Ad esempio, per catturare tutto il traffico sulla porta 80:
    ```bash
    sudo tcpdump -i eth0 -w cattura_http.pcap 'port 80'
    ```
    Premi `Ctrl+C` per terminare la cattura. Il file `cattura_http.pcap` conterrà i pacchetti.

### Windows (con Wireshark)

Wireshark offre un'interfaccia grafica intuitiva:

1.  Scarica e installa Wireshark dal sito ufficiale.
2.  Avvia Wireshark. Vedrai una lista delle interfacce di rete.
3.  Fai doppio clic sull'interfaccia che sta generando traffico (es. "Wi-Fi" o "Ethernet").
4.  La cattura si avvierà automaticamente. Per fermarla, clicca sul pulsante rosso a forma di quadrato ("Stop").
5.  Per salvare, vai su `File > Save As...` e scegli il formato `.pcap`.

In [ ]:
## 5. Attività Pratica 2: Creazione di un File .pcap Fittizio

# A volte è utile creare pacchetti artificiali per testare script di analisi in modo controllato.
# Useremo la libreria Scapy per questo. Se non è installata, esegui: !pip install scapy

try:
    from scapy.all import *
    print("Scapy importato correttamente.")
except ImportError:
    print("Scapy non trovato. Installazione in corso...")
    import os
    os.system('pip install scapy')
    from scapy.all import *

# 1. Creiamo un pacchetto TCP SYN
# IP: Sorgente 192.168.1.10, Destinazione 1.1.1.1
# TCP: Porta sorgente 1234, Porta destinazione 443 (HTTPS), Flag SYN, Sequence Number 1000
ip_layer_tcp = IP(src="192.168.1.10", dst="1.1.1.1")
tcp_layer = TCP(sport=1234, dport=443, flags="S", seq=1000)
packet_tcp_syn = ip_layer_tcp / tcp_layer

# 2. Creiamo un pacchetto UDP (query DNS)
# IP: Sorgente 192.168.1.10, Destinazione 8.8.8.8 (Google DNS)
# UDP: Porta destinazione 53 (DNS)
ip_layer_udp = IP(src="192.168.1.10", dst="8.8.8.8")
udp_layer = UDP(dport=53)
dns_layer = DNS(rd=1, qd=DNSQR(qname="www.google.com"))
packet_udp_dns = ip_layer_udp / udp_layer / dns_layer

# 3. Mettiamo insieme i pacchetti e salviamoli in un file .pcap
packets = [packet_tcp_syn, packet_udp_dns]
wrpcap("simulazione_header.pcap", packets)

print("File 'simulazione_header.pcap' creato con successo!")
print("\n--- Dettagli Pacchetto TCP SYN ---")
packet_tcp_syn.show()
print("\n--- Dettagli Pacchetto UDP DNS ---")
packet_udp_dns.show()

In [ ]:
## 6. Attività Pratica 3: Analisi Programmatica degli Header

# Ora leggiamo il file .pcap e analizziamo i pacchetti che contiene.
# Completa il codice sostituendo i placeholder [BLANK_X] con le istruzioni corrette.

from scapy.all import *

file_pcap = "simulazione_header.pcap"
pacchetti_letti = rdpcap(file_pcap)

print(f"Trovati {len(pacchetti_letti)} pacchetti nel file.\n")

for i, pkt in enumerate(pacchetti_letti):
    print(f"--- Analisi Pacchetto #{i+1} ---")
    
    # Controlliamo se il pacchetto ha un layer IP
    if pkt.haslayer(IP):
        ip_src = pkt[IP].src
        ip_dst = pkt[IP].dst
        print(f"  [IP] Sorgente: {ip_src}, Destinazione: {ip_dst}")
        
        # Se è un pacchetto TCP, analizza l'header TCP
        if pkt.haslayer(TCP):
            # [BLANK_1]: Estrai la porta sorgente TCP
            tcp_sport = '[BLANK_1]'
            # [BLANK_2]: Estrai la porta destinazione TCP
            tcp_dport = '[BLANK_2]'
            # [BLANK_3]: Estrai il sequence number
            tcp_seq = '[BLANK_3]'
            # [BLANK_4]: Estrai i flag TCP (es. 'S' per SYN)
            tcp_flags = '[BLANK_4]'
            
            print(f"    [TCP] Porta Sorgente: {tcp_sport}")
            print(f"    [TCP] Porta Destinazione: {tcp_dport}")
            print(f"    [TCP] Sequence Number: {tcp_seq}")
            print(f"    [TCP] Flags: {tcp_flags}")

        # Se è un pacchetto UDP, analizza l'header UDP
        elif pkt.haslayer(UDP):
            # [BLANK_5]: Estrai la porta destinazione UDP
            udp_dport = '[BLANK_5]'
            print(f"    [UDP] Porta Destinazione: {udp_dport}")
            
            if pkt.haslayer(DNS):
                qname = pkt[DNSQR].qname.decode()
                print(f"      [DNS] Query per: {qname}")

## 7. Autovalutazione e Domande Teoriche

Rispondi alle seguenti domande per verificare la tua comprensione dei concetti avanzati.

**Quesito 1:** In quale scenario pratico un programmatore di rete sceglierebbe di usare `shutdown(socket.SHUT_WR)` invece di un semplice `close()`? Spiega il vantaggio di questa scelta.

**Quesito 2:** Perché la finestra di congestione iniziale di TCP (durante il Slow Start) è volutamente piccola invece di essere impostata al massimo valore possibile? Quale problema di rete si cerca di prevenire?

## 8. Soluzioni per l'Autovalutazione

### Risposte ai Quesiti Teorici

**Risposta Quesito 1:**
`shutdown(socket.SHUT_WR)` è ideale in uno scenario client-server dove il client invia una richiesta di lunghezza variabile e vuole segnalare al server che ha finito di trasmettere dati, ma deve ancora ricevere la risposta del server. Ad esempio, in un protocollo HTTP/1.0, dopo aver inviato una richiesta `POST`, il client può chiudere il canale di scrittura. Il server, vedendo la chiusura (ricevendo 0 byte), sa che la richiesta è completa e può iniziare a processarla e inviare la risposta. Il client, nel frattempo, mantiene il canale di lettura aperto per ricevere tale risposta. Usare `close()` chiuderebbe immediatamente entrambi i canali, rendendo impossibile ricevere la risposta del server.

**Risposta Quesito 2:**
La finestra di congestione iniziale è piccola per prevenire il **sovraccarico della rete (congestion)**. All'inizio di una connessione, il mittente non ha informazioni sulla capacità del percorso di rete (es. banda disponibile, latenza, presenza di colli di bottiglia). Iniziare inviando una grande quantità di dati potrebbe immediatamente congestionare un link con poca capacità, causando perdita di pacchetti e un degrado delle performance per tutti. Il meccanismo di **Slow Start** agisce come una sonda: partendo piano e aumentando esponenzialmente la velocità solo in presenza di ACK positivi, TCP si adatta dinamicamente alla capacità effettiva della rete, garantendo un utilizzo efficiente e sicuro delle risorse.

### Soluzione per l'Esercizio di Analisi Pacchetti (`[BLANK_X]`)


In [ ]:
# Soluzione per i placeholder [BLANK_X]

# [BLANK_1]: pkt[TCP].sport
# [BLANK_2]: pkt[TCP].dport
# [BLANK_3]: pkt[TCP].seq
# [BLANK_4]: pkt[TCP].flags
# [BLANK_5]: pkt[UDP].dport

# Codice completo e corretto:
from scapy.all import *

file_pcap = "simulazione_header.pcap"
pacchetti_letti = rdpcap(file_pcap)

print(f"Trovati {len(pacchetti_letti)} pacchetti nel file.\n")

for i, pkt in enumerate(pacchetti_letti):
    print(f"--- Analisi Pacchetto #{i+1} ---")
    if pkt.haslayer(IP):
        ip_src = pkt[IP].src
        ip_dst = pkt[IP].dst
        print(f"  [IP] Sorgente: {ip_src}, Destinazione: {ip_dst}")
        if pkt.haslayer(TCP):
            tcp_sport = pkt[TCP].sport
            tcp_dport = pkt[TCP].dport
            tcp_seq = pkt[TCP].seq
            tcp_flags = pkt[TCP].flags
            print(f"    [TCP] Porta Sorgente: {tcp_sport}")
            print(f"    [TCP] Porta Destinazione: {tcp_dport}")
            print(f"    [TCP] Sequence Number: {tcp_seq}")
            print(f"    [TCP] Flags: {tcp_flags}")

        elif pkt.haslayer(UDP):
            udp_dport = pkt[UDP].dport
            print(f"    [UDP] Porta Destinazione: {udp_dport}")
            if pkt.haslayer(DNS):
                qname = pkt[DNSQR].qname.decode()
                print(f"      [DNS] Query per: {qname}")